In [3]:
import os
import pandas as pd

## Tables

In [88]:
dir_path = os.path.join("results", "ihdp")
setups = ["samePOihdp_GC","samePOihdp-highhc_GC","samePOihdp-lowhc_GC","diffPOihdp-highdiff_GC","diffPOihdp-lowdiff_GC"]

setups_names_dict = {"samePOihdp_GC": "Same PO, GC",
                     "samePOihdp-highhc_GC": "Same PO, GC, high HC",
                     "samePOihdp-lowhc_GC": "Same PO, GC, low HC",
                     "diffPOihdp-highdiff_GC": "Diff PO, GC, high diff",
                     "diffPOihdp-lowdiff_GC": "Diff PO, GC, low diff"}

In [89]:
def recursive_results(path,dirlist,str_, res_):
    if "res_summary.csv" in dirlist:
        res = pd.read_csv(os.path.join(path,"res_summary.csv"))
        return [res], str_
    else:
        r_tmp = res_.copy()
        s_tmp = []
        for d in dirlist:
            path_ = os.path.join(path,d)
            r_, s_ = recursive_results(path_, os.listdir(path_), [str_[0] + "-*-" + d],res_)
            r_tmp = r_tmp + r_
            s_tmp = s_tmp + s_
        return r_tmp, s_tmp


In [97]:
df_list = []
for setup in setups:
    dir_ = os.path.join(dir_path, setup)
    dirlist = os.listdir(dir_)
    r_list, s_list = recursive_results(dir_,dirlist,[""],[]) 

    assert len(r_list) == len(s_list)

    df_list_ = []
    for i in range(len(r_list)):
        p_val = r_list[i].iloc[0][1].item()
        config = s_list[i].split("-*-")[1:]
        config_dict = {"Signal":config[0].split("'")[1].replace("_","-"), "M": config[1].split('[')[1].split(']')[0] ,"CD":config[2], "UC":config[3], "p-value":p_val}
        df_list_.append(pd.DataFrame(config_dict, index = [i]))
    df_ = pd.concat(df_list_)
    df_.sort_values(by = ["Signal","M","CD","UC"], inplace = True)
    #df_["setup"] = setup
    df_list.append(df_)
df = pd.concat(df_list, axis = 1, keys = [setups_names_dict[setup] for setup in setups])

/tmp/ipykernel_3071818/2784463539.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_val = r_list[i].iloc[0][1].item()
/tmp/ipykernel_3071818/2784463539.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_val = r_list[i].iloc[0][1].item()
/tmp/ipykernel_3071818/2784463539.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  p_val = r_list[i].iloc[0][1].item()
/tmp/ipykernel_3071818/2784463539.py:11: FutureWarning: Series.__ge

In [98]:
df

Same PO, GC                         Same PO, GC, high HC                  \
       Signal  M     CD    UC p-value               Signal  M     CD    UC   
2         CDR  1  CD-10  UC-0   0.625                  CDR  1  CD-10  UC-1   
5     DR-Drop  1  CD-10  UC-0   0.400              DR-Drop  1  CD-10  UC-1   
0   DR-Impute  1  CD-10  UC-0   0.200            DR-Impute  1  CD-10  UC-1   
3        IPCW  1  CD-10  UC-0   0.000                 IPCW  1  CD-10  UC-1   
6     IPCW-GC  1  CD-10  UC-0   0.000              IPCW-GC  1  CD-10  UC-1   
4    IPW-Drop  1  CD-10  UC-0   0.075             IPW-Drop  1  CD-10  UC-1   
1  IPW-Impute  1  CD-10  UC-0   0.025           IPW-Impute  1  CD-10  UC-1   

           ... Diff PO, GC, high diff                          \
  p-value  ...                 Signal  M     CD    UC p-value   
2   0.775  ...                    CDR  1  CD-10  UC-0   0.925   
5   1.000  ...                DR-Drop  1  CD-10  UC-0   1.000   
0   1.000  ...              DR-Impute  1  CD-10  UC-0   1.000   
3   0.000  ...                   IPCW  1  CD-10  UC-0   0.175   
6   0.875  ...                IPCW-GC  1  CD-10  UC-0   0.725   
4   0.875  ...               IPW-Drop  1  CD-10  UC-0   0.875   
1   0.950  ...             IPW-Impute  1  CD-10  UC-0   0.975   

  Diff PO, GC, low diff                          
                 Signal  M     CD    UC p-value  
2                   CDR  1  CD-10  UC-0   0.225  
5               DR-Drop  1  CD-10  UC-0   0.375  
0             DR-Impute  1  CD-10  UC-0   0.350  
3                  IPCW  1  CD-10  UC-0   0.000  
6               IPCW-GC  1  CD-10  UC-0   0.000  
4              IPW-Drop  1  CD-10  UC-0   0.000  
1            IPW-Impute  1  CD-10  UC-0   0.000  

[7 rows x 25 columns]

In [100]:
print(df.to_latex(escape = False,index = False))

\begin{tabular}{llllrllllrllllrllllrllllr}
\toprule
\multicolumn{5}{r}{Same PO, GC} & \multicolumn{5}{r}{Same PO, GC, high HC} & \multicolumn{5}{r}{Same PO, GC, low HC} & \multicolumn{5}{r}{Diff PO, GC, high diff} & \multicolumn{5}{r}{Diff PO, GC, low diff} \\
Signal & M & CD & UC & p-value & Signal & M & CD & UC & p-value & Signal & M & CD & UC & p-value & Signal & M & CD & UC & p-value & Signal & M & CD & UC & p-value \\
\midrule
CDR & 1 & CD-10 & UC-0 & 0.625000 & CDR & 1 & CD-10 & UC-1 & 0.775000 & CDR & 1 & CD-10 & UC-1 & 0.800000 & CDR & 1 & CD-10 & UC-0 & 0.925000 & CDR & 1 & CD-10 & UC-0 & 0.225000 \\
DR-Drop & 1 & CD-10 & UC-0 & 0.400000 & DR-Drop & 1 & CD-10 & UC-1 & 1.000000 & DR-Drop & 1 & CD-10 & UC-1 & 0.625000 & DR-Drop & 1 & CD-10 & UC-0 & 1.000000 & DR-Drop & 1 & CD-10 & UC-0 & 0.375000 \\
DR-Impute & 1 & CD-10 & UC-0 & 0.200000 & DR-Impute & 1 & CD-10 & UC-1 & 1.000000 & DR-Impute & 1 & CD-10 & UC-1 & 0.775000 & DR-Impute & 1 & CD-10 & UC-0 & 1.000000 & DR-Impute & 1 